#목표


#데이터 살펴보기


In [6]:
#프로젝트를 진행함에 필요한 모듈을 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#파이널프로젝트_RAW_210329_210926.xlsx 마운팅 후 DataFrame으로 불러오기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df=pd.read_excel('/content/drive/MyDrive/미니프로젝트/파이널프로젝트_RAW_210329_210926.xlsx')
df.head()

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at
0,970634,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,110500.0,NaN,VBANK,개발자 커리어,NaT
1,970650,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
2,970657,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
3,970656,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
4,970658,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14


| 컬럼 | 설명 |
| ---- | ----|
| (삭제) id | << 버려도 됨..??? |
| customer_id | 중복있음!! +순수 고객수 파악하려면 전처리 필요 |
| course_id | 어떤 강의 듣는지(빈 값은 폐강으로 추측) |
| type    | 해당 강좌의 현재 결제 진행상태(order는 장바구니, payment가 최종 결제) |
| state                   | type의 진행상황 |
| course_title            | 강의 제목 |
| category_title          | 강의 카테고리 제목 |
| (삭제 보류)format         | 강의 과정 분류 종류|
| completed_at            | 강의 결제 완료 시각|
| transaction_amount      | 실 결제 금액|
| coupon_title            | 쿠폰명|
| coupon_discount_amount  | 할인된 금액|
| sale_price              | 판매 금액|
| (삭제) tax_free_amount    | 면세액 |
| pg                      | 결제대행사 (빈칸은 그 외의 결제대행사)|
| method                  | 결제 방법|
| subcategory_title       | 강의 부 카테고리 제목|
| marketing_start_at      | 해당 강의가 열린날 (일단위로 기록됨)|


____
<유의할 사항>
- 쿠폰 말고도 이벤트같은 다른 조건에 의한 할인도 있다


#데이터 타입과 결측값 확인하기

>결측치 제거

In [11]:
#결측치 확인
df = df.dropna(subset=['category_title'])
df.isna().sum()

id                            0
customer_id                   0
course_id                  5183
type                          0
state                         0
course_title                  0
category_title                0
format                        0
completed_at                  0
transaction_amount            0
coupon_title              73578
coupon_discount_amount    73578
sale_price                 1112
tax_free_amount            1112
pg                        15058
method                    13095
subcategory_title         19805
marketing_start_at        17955
dtype: int64

id는 단순히 자료 처리 순서이므로 삭제한다.

tax_free_amount는 보안상의 이유로 삭제한다.

In [12]:
# 'id'와'tax_free_amount' 컬럼 제거
df = df.drop(['id','tax_free_amount'],axis = 'columns')
df.head()

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,NaN,VBANK,개발자 커리어,NaT
1,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,NaN,CARD,영상 편집,2020-12-02
2,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
3,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,NaN,CARD,NaN,2021-03-04
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14


>결제 완료한 사람 중 환불하지 않은 사람만 판별 

In [13]:
# 결제가 완료된 사람의 데이터
df_pc = df[(df['type'] == 'PAYMENT') & (df['state'] == 'COMPLETED')]
df_pc.head(3)

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
2,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
8,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02


In [14]:
# 환불이 완료된 사람의 데이터
df_rc =df[(df['type'] == 'REFUND') & (df['state'] == 'COMPLETED')]
df_rc.head(3)

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
651,444760.0,204122.0,REFUND,COMPLETED,퍼포먼스 마케터 되기 온라인 완주반 2기,마케팅,온라인 완주반,2021-03-29 09:24:43,-147000,NaN,NaN,404000.0,NaN,VBANK,NaN,NaT
652,472666.0,203178.0,REFUND,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-29 09:26:13,-96000,NaN,NaN,116000.0,NaN,CARD,영상 편집,2020-12-02
666,447177.0,203989.0,REFUND,COMPLETED,올인원 패키지 : 소금툰과 함께 캐릭터 드로잉/굿즈/임티/컷툰,디자인,올인원,2021-03-29 09:40:54,-79000,NaN,NaN,89000.0,NaN,CARD,일러스트,2021-02-19


In [15]:
# 결제가 완료된 사람과 환불이 완료된 사람의 데이터를 합쳐준다.
df_total = pd.merge(df_pc,df_rc,how='outer')
df_total.head(3)

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
1,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
2,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02


In [16]:
# 합친 데이터 중에서 중복된 customer_id를 제거해준다.
df_total= df_total.drop_duplicates(subset='customer_id')
df_total['type'].value_counts()

PAYMENT    39516
REFUND      1049
Name: type, dtype: int64

In [17]:
# 남아있는 REFUND는 제공된 데이터 기간 전에 결제를 완료하고, 제공된 기간 내에 환불이 완료된 데이터이다.
# 정해진 기간내의 환불 비율을 알고싶으므로 PAYMENT만 골라낸다.
df_pay = df_total['type'] == 'PAYMENT'
df_pay = df_total[df_pay]
df_pay['type'].value_counts()

PAYMENT    39516
Name: type, dtype: int64

>카테고리 컬럼 수정


In [18]:
df['category_title'].value_counts().to_frame()

# 수정 목록
# 1. 띄어쓰기 차이로 다른 강의로 구분된 '데이터사이언스'와 '데이터 사이언스'를 합쳐준다.
# 2. 디자인 카테고리에 서브 카테고리로 속해있는 '일러스트' 카테고리를 '디자인'카테고리에 편입시켜준다.
# 3. '교육','지식콘텐츠','비즈니스','파이낸스','관리' 카테고리는 현재 폐강된 카테고리들로 삭제해준다.

,category_title
프로그래밍,34430
데이터사이언스,14076
업무 생산성,13323
디자인,12009
영상/3D,10260
마케팅,9857
부동산/금융,5523
투자/재테크,2501
크리에이티브,2250
교육,1031


In [19]:
# 카테고리 변경
df_cate = df.replace({'category_title':{'데이터 사이언스':'데이터사이언스'}})
df_cate = df_cate.replace({'category_title':{'일러스트':'디자인'}})
df_cate['category_title'].value_counts().to_frame()

,category_title
프로그래밍,34430
데이터사이언스,14085
업무 생산성,13323
디자인,12010
영상/3D,10260
마케팅,9857
부동산/금융,5523
투자/재테크,2501
크리에이티브,2250
교육,1031


In [20]:
# 불필요한 카테고리 삭제
df_cate = df_cate[df_cate.category_title != '교육']
df_cate = df_cate[df_cate.category_title != '지식콘텐츠']
df_cate = df_cate[df_cate.category_title != '비즈니스']
df_cate = df_cate[df_cate.category_title != '파이낸스']
df_cate = df_cate[df_cate.category_title != '관리']

df_cate['category_title'].value_counts()

프로그래밍      34430
데이터사이언스    14085
업무 생산성     13323
디자인        12010
영상/3D      10260
마케팅         9857
부동산/금융      5523
투자/재테크      2501
크리에이티브      2250
Name: category_title, dtype: int64

In [21]:
#웰컴쿠폰 사용자 통합
df[(df['coupon_title'].str.contains('웰컴')) | (df['coupon_title'].str.contains('WELCOME'))]

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
8,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
14,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
15,478035.0,203720.0,ORDER,CANCELLED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:02,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
28,471759.0,204068.0,PAYMENT,COMPLETED,올인원 패키지 : 마디아의 UX/UI 포트폴리오,디자인,올인원,2021-03-28 15:46:22,89000,[웰컴쿠폰] 올인원 패키지 1만원 할인 쿠폰,10000.0,99000.0,NaN,CARD,UX/UI,2021-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105393,638903.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-09-25 11:22:07,139000,[WELCOME] 온라인 강의 1만원 할인쿠폰,10000.0,149000.0,KAKAO,POINT,영상 편집,2020-12-02
105396,623392.0,206060.0,PAYMENT,CANCELLED,초격차 패키지 : 한 번에 끝내는 컴퓨터 공학 전공필수 & 인공지능 심화,프로그래밍,올인원,2021-09-25 11:38:53,173000,[WELCOME] 온라인 강의 3만원 할인쿠폰,30000.0,203000.0,KAKAO,NaN,개발자 커리어,2021-07-16
105414,640697.0,205365.0,PAYMENT,CANCELLED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:10:42,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,INICIS,TRANS,영상 편집,2021-06-15
105416,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15


In [22]:
df_pay[(df_pay['transaction_amount'] == 0) & (df_pay['coupon_title'].str.contains('검수'))]

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
337,394318.0,204159.0,PAYMENT,COMPLETED,부동산 디벨로퍼 아카데미 : 인허가,부동산/금융,올인원,2021-03-29 10:43:18,0,[무료수강권] 강의 검수용_이호상 PM님,166000.0,166000.0,NaN,PROMOTION,부동산 개발,2021-02-10 00:00:00
571,158374.0,204150.0,PAYMENT,COMPLETED,올인원 패키지 : 십다 프리미어,영상/3D,올인원,2021-03-29 16:48:47,0,crm 검수용 쿠폰 aop,79000.0,79000.0,NaN,PROMOTION,영상 편집,2021-02-26 00:00:00
747,469897.0,203787.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 디자인 툴,디자인,올인원,2021-03-29 21:56:00,0,[무료수강권] 강의검수용_강사님 쿠폰_박혜린 강사님,170000.0,170000.0,NaN,PROMOTION,디자인툴,2020-12-24 11:00:00
949,3189.0,203146.0,PAYMENT,COMPLETED,The RED : 현실 세상의 TDD by 이규원,프로그래밍,RED,2021-03-30 10:50:38,0,[무료수강권] 내부직원 검수용 RED,359000.0,359000.0,NaN,PROMOTION,DevOps,2021-02-08 00:00:00
981,480408.0,202785.0,PAYMENT,COMPLETED,올인원 패키지 : 인스타를 성장시킨 50명의 성공방정식,마케팅,올인원,2021-03-30 12:13:46,0,[무료수강권] 올인원 패키지 : 인스타를 성장시킨 50명의 성공 방정식 강의검수용_...,80000.0,80000.0,NaN,PROMOTION,SNS마케팅,2021-01-18 00:00:00
6098,178237.0,204342.0,PAYMENT,COMPLETED,"올인원 패키지 : GA4 탑재, 구글 애널리틱스 끝장내기",마케팅,올인원,2021-04-16 15:34:15,0,[무료수강권] AOP 강의 검수용_어효경님,79000.0,79000.0,NaN,PROMOTION,데이터마케팅,2021-04-02 00:00:00
6918,502517.0,202785.0,PAYMENT,COMPLETED,올인원 패키지 : 인스타를 성장시킨 50명의 성공방정식,마케팅,올인원,2021-04-20 18:14:25,0,[무료수강권] 올인원 패키지 : 인스타를 성장시킨 50명의 성공 방정식 강의검수용_...,80500.0,80500.0,NaN,PROMOTION,SNS마케팅,2021-01-18 00:00:00
7285,30445.0,203989.0,PAYMENT,COMPLETED,올인원 패키지 : 소금툰과 함께 캐릭터 드로잉/굿즈/임티/컷툰,디자인,올인원,2021-04-22 19:04:18,0,[무료수강권] 강의 검수용 무료 수강권 (패캐머용),91000.0,91000.0,NaN,PROMOTION,일러스트,2021-02-19 00:00:00
7401,450281.0,203535.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 PPT 제작/디자인,업무 생산성,올인원,2021-04-23 11:24:46,0,예스폼 검수용 쿠폰,100000.0,100000.0,NaN,PROMOTION,PPT/보고서,2021-03-10 00:00:00
9239,425706.0,203533.0,PAYMENT,COMPLETED,연습문제 패키지 : 퀴즈처럼 풀면서 배우는 파이썬 머신러닝 300제+,데이터사이언스,올인원,2021-04-28 11:27:14,0,[무료수강권] 강의 검수용_조은주 매니저님,82000.0,82000.0,NaN,PROMOTION,머신러닝,2021-01-05 00:00:00


In [23]:
df_welcome_group = df_pay[df_pay['coupon_title'].str.contains('WELCOME') | df_pay['coupon_title'].str.contains('웰컴')]

In [24]:
df_welcome_group

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
1,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
2,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
4,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
7,471759.0,204068.0,PAYMENT,COMPLETED,올인원 패키지 : 마디아의 UX/UI 포트폴리오,디자인,올인원,2021-03-28 15:46:22,89000,[웰컴쿠폰] 올인원 패키지 1만원 할인 쿠폰,10000.0,99000.0,NaN,CARD,UX/UI,2021-03-11
14,445723.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 16:15:46,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52772,604077.0,205569.0,PAYMENT,COMPLETED,올인원 패키지 : 공여사와 무작정 풀어보는 엑셀 실무,업무 생산성,올인원,2021-09-24 16:49:27,109000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,129000.0,KAKAO,POINT,엑셀/VBA,2021-08-17
52856,646968.0,206888.0,PAYMENT,COMPLETED,올인원 패키지 : 글로벌 1위 Salesforce 디자인 디렉터의 B2B SaaS ...,디자인,올인원,2021-09-25 02:37:22,149000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,169000.0,NAVER,CARD,UX/UI,2021-08-23
52879,638903.0,206720.0,PAYMENT,COMPLETED,용호수의 돈 버는 실전 영상 제작,영상/3D,올인원,2021-09-25 10:56:03,159000,[WELCOME] 온라인 강의 1만원 할인쿠폰,10000.0,169000.0,KAKAO,POINT,영상 편집,2021-08-17
52897,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15


#데이터 분포 확인하기

할인금액

- 얼마 할인해줬을 때, 가장 많은 결제를 이끌어냈는지

- 같은 카테고리기준으로 할인 혜택을 받은 사람과 안받은 사람의 비율



In [48]:
#할인금액 분석에서 사용할 연습용 데이터 프레임 생성
df_pay2 = df_pay[['customer_id','sale_price','transaction_amount','coupon_discount_amount']]

In [49]:
#계산시 오류나지 않게 transaction_amount를 float값으로 변환
df_pay2 = df_pay2.astype({'transaction_amount':'float'})

In [50]:
# 얼마 할인받는지 계산
df_pay2['discounted_amount'] = df_pay2['sale_price'] - df_pay2['transaction_amount']
df_pay2

,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount
0,72297.0,190900.0,171810.0,19090.0,19090.0
1,478028.0,117500.0,97500.0,20000.0,20000.0
2,478033.0,116000.0,96000.0,20000.0,20000.0
3,469860.0,263000.0,263000.0,NaN,0.0
4,478035.0,111000.0,91000.0,20000.0,20000.0
...,...,...,...,...,...
52893,545804.0,1800000.0,1800000.0,NaN,0.0
52894,669238.0,232000.0,212000.0,20000.0,20000.0
52896,670891.0,179000.0,159000.0,20000.0,20000.0
52897,640697.0,116000.0,96000.0,20000.0,20000.0


In [51]:
#NaN값 0.0으로 변경
df_pay2['coupon_discount_amount'] = df_pay2['coupon_discount_amount'].replace(np.NaN,0.0)
df_pay2

,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount
0,72297.0,190900.0,171810.0,19090.0,19090.0
1,478028.0,117500.0,97500.0,20000.0,20000.0
2,478033.0,116000.0,96000.0,20000.0,20000.0
3,469860.0,263000.0,263000.0,0.0,0.0
4,478035.0,111000.0,91000.0,20000.0,20000.0
...,...,...,...,...,...
52893,545804.0,1800000.0,1800000.0,0.0,0.0
52894,669238.0,232000.0,212000.0,20000.0,20000.0
52896,670891.0,179000.0,159000.0,20000.0,20000.0
52897,640697.0,116000.0,96000.0,20000.0,20000.0


In [52]:
#쿠폰 할인외에 이벤트 할인받은 데이터가 있는지 확인
aa = df_pay2['coupon_discount_amount'] == df_pay2['discounted_amount']
aa.value_counts() 

#쿠폰 할인 받은 사람 + 쿠폰 할인 못 받은 사람 : 36700 명
#이벤트 할인 받은 사람: 2819 명

True     36698
False     2818
dtype: int64

In [53]:
bb = df_pay2['coupon_discount_amount'] != 0 
bb.value_counts()

#쿠폰 할인 받은 사람: 17134 명
#쿠폰 할인 못받은 사람: 36700 - 17134 = 19566 명

False    22382
True     17134
Name: coupon_discount_amount, dtype: int64

In [54]:
#할인 타입 구분 
is_not_discount = (df_pay2['coupon_discount_amount'] == df_pay2['discounted_amount']) & (df_pay2['coupon_discount_amount'] == 0)
is_coupon = (df_pay2['coupon_discount_amount'] == df_pay2['discounted_amount']) & (df_pay2['coupon_discount_amount'] != 0)
is_event = (df_pay2['coupon_discount_amount'] != df_pay2['discounted_amount']) & (df_pay2['coupon_discount_amount'] != 0)


df_pay2

,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount
0,72297.0,190900.0,171810.0,19090.0,19090.0
1,478028.0,117500.0,97500.0,20000.0,20000.0
2,478033.0,116000.0,96000.0,20000.0,20000.0
3,469860.0,263000.0,263000.0,0.0,0.0
4,478035.0,111000.0,91000.0,20000.0,20000.0
...,...,...,...,...,...
52893,545804.0,1800000.0,1800000.0,0.0,0.0
52894,669238.0,232000.0,212000.0,20000.0,20000.0
52896,670891.0,179000.0,159000.0,20000.0,20000.0
52897,640697.0,116000.0,96000.0,20000.0,20000.0


In [56]:
if is_not_discount == True:
  df['discount_type'] =  'x'
elif is_coupon == True:
  df['discount_type'] = 'coupon'
elif is_event == True:
  df['discount_type'] = 'event'

ValueError: ignored

In [47]:
df_pay2['discount_type'] 
df_pay2

,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount,discount_type
0,72297.0,190900.0,171810.0,19090.0,19090.0,True
1,478028.0,117500.0,97500.0,20000.0,20000.0,True
2,478033.0,116000.0,96000.0,20000.0,20000.0,True
3,469860.0,263000.0,263000.0,0.0,0.0,True
4,478035.0,111000.0,91000.0,20000.0,20000.0,True
...,...,...,...,...,...,...
52893,545804.0,1800000.0,1800000.0,0.0,0.0,True
52894,669238.0,232000.0,212000.0,20000.0,20000.0,True
52896,670891.0,179000.0,159000.0,20000.0,20000.0,True
52897,640697.0,116000.0,96000.0,20000.0,20000.0,True


재결제율

- 웰컴 할인쿠폰을 받은 사람 중 재결제한사람(신규 유입에 쿠폰의 영향?)

- 동일한 결제 수단으로 결제 여부

>최적의 가성비 할인율은 얼마?

판매 금액의 얼마정도를 할인했을 때 결제율이 상승하는지, 할인율을 10% 단위로 분류해서 비교

<추가 분석>

1. 장바구니에서 최종 결제까지 가기에 할인쿠폰/이벤트가 영향을 주는가?

2. 해비 유저와 라이트 유저 (평균값 기준으로 구분) 둘 중 누구에게 할인쿠폰/이벤트의 영향이 큰지 >> 두 그룹 중 어느 그룹이 매출에 큰 영향을 끼치는지